# Script Assignment 6 - Group 1
#### Marco Bogani, Daman Sawhney, Sulaiman Karmali, Ivan Lima, Zach Novak

In Financial Technology (FinTech), anomaly detection plays a crucial role in identifying fraudulent activities such as money laundering or unauthorized transactions. One popular method for anomaly detection is DBSCAN (Density-Based Spatial Clustering of Applications with Noise), which effectively identifies clusters of data points that are closely packed together while marking outliers as noise. In this assignment, you are tasked with implementing a Python script to detect anomalous transactions in financial data using DBSCAN.

Instructions:
1. Download the provided financial transaction datasetLinks to an external site.. Links to an external site.Preprocess the data if necessary (e.g., normalization, feature engineering). (5 points)

2. Implement the DBSCAN algorithm using the scikit-learn library. Tune the hyperparameters such as epsilon (eps) and minimum samples (min_samples) to achieve optimal performance. Cluster the transactions using DBSCAN.  (10 points)

3. Identify anomalous transactions as outliers detected by DBSCAN. Analyze the characteristics of these anomalies to understand their nature. Visualize the clusters and outliers for better insights.  (5 points)

Import Libraries

In [ ]:
import pandas as pd